In [4]:
pip install pulp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 28.6 MB/s eta 0:00:00


QUESTION 1

---




In [6]:

from pulp import LpProblem, LpMinimize, LpVariable, lpSum

# Create a problem variable
problem_transportation = LpProblem("Warehouse_MarketOptimization", LpMinimize)

# Define the warehouses, fruits, markets, and distances
warehouses = ['A', 'B', 'C', 'D']
markets = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'M10']
fruits = ['Lemon', 'Tangerine', 'Orange', 'Apple', 'Banana']
distances = {
    ('A', 'M1'): 2, ('A', 'M2'): 5, ('A', 'M3'): 4, ('A', 'M4'): 3, ('A', 'M5'): 4, ('A', 'M6'): 3, ('A', 'M7'): 1, ('A', 'M8'): 2, ('A', 'M9'): 6, ('A', 'M10'): 3,
    ('B', 'M1'): 5, ('B', 'M2'): 3, ('B', 'M3'): 4, ('B', 'M4'): 2, ('B', 'M5'): 5, ('B', 'M6'): 4, ('B', 'M7'): 6, ('B', 'M8'): 4, ('B', 'M9'): 2, ('B', 'M10'): 1,
    ('C', 'M1'): 3, ('C', 'M2'): 4, ('C', 'M3'): 2, ('C', 'M4'): 5, ('C', 'M5'): 4, ('C', 'M6'): 4, ('C', 'M7'): 2, ('C', 'M8'): 5, ('C', 'M9'): 7, ('C', 'M10'): 2,
    ('D', 'M1'): 4, ('D', 'M2'): 2, ('D', 'M3'): 1, ('D', 'M4'): 1, ('D', 'M5'): 2, ('D', 'M6'): 3, ('D', 'M7'): 5, ('D', 'M8'): 3, ('D', 'M9'): 5, ('D', 'M10'): 2,
}

# Define the number of pieces that fit in a case for each fruit
capacity = {
    'Lemon': 50,
    'Tangerine': 40,
    'Orange': 20,
    'Apple': 30,
    'Banana': 20,
}

# Define the demand for each market
demand = {
    'M1': {'Lemon': 150, 'Tangerine': 80, 'Orange': 80, 'Apple': 0, 'Banana': 0},
    'M2': {'Lemon': 200, 'Tangerine': 200, 'Orange': 200, 'Apple': 30, 'Banana': 40},
    'M3': {'Lemon': 0, 'Tangerine': 120, 'Orange': 100, 'Apple': 60, 'Banana': 0},
    'M4': {'Lemon': 0, 'Tangerine': 0, 'Orange': 0, 'Apple': 150, 'Banana': 100},
    'M5': {'Lemon': 200, 'Tangerine': 200, 'Orange': 0, 'Apple': 0, 'Banana': 0},
    'M6': {'Lemon': 100, 'Tangerine': 80, 'Orange': 100, 'Apple': 120, 'Banana': 0},
    'M7': {'Lemon': 0, 'Tangerine': 0, 'Orange': 0, 'Apple': 300, 'Banana': 200},
    'M8': {'Lemon': 0, 'Tangerine': 0, 'Orange': 0, 'Apple': 300, 'Banana': 100},
    'M9': {'Lemon': 100, 'Tangerine': 0, 'Orange': 0, 'Apple': 60, 'Banana': 0},
    'M10': {'Lemon': 150, 'Tangerine': 0, 'Orange': 0, 'Apple': 0, 'Banana': 20},
}
# Define decision variables
x = LpVariable.dicts("Cases", (warehouses, fruits, markets), 0, None, cat = 'Integer')

y = LpVariable.dicts("Assign", (warehouses, markets), cat = 'Binary')

# Define the objective function
problem_transportation += lpSum(distances[(w, m)] * x[w][f][m] for w in warehouses for f in fruits  for m in markets)

# Demand constraints
for m in markets:
    for f in fruits:
        problem_transportation += lpSum(x[w][f][m]*capacity[f] for w in warehouses) == demand[m][f]

# Capacity constraints
for w in warehouses:
     problem_transportation += lpSum(x[w][f][m] for f in fruits for m in markets) <= 30


# Specific warehouse constraints
for m in markets:
    problem_transportation += lpSum(x['D'][f][m] for f in ['Lemon', 'Tangerine', 'Orange']) == 0  # No Lemons, Tangerines, or Oranges in warehouse D
    problem_transportation += x['A']['Banana'][m] == 0  # No Bananas in warehouse A

# Each market can get service from only one warehouse
for m in markets:
    problem_transportation += lpSum(y[w][m] for w in warehouses) == 1

# Additional constraint: If a warehouse serves a market, the number of cases of each fruit should be less than or equal to 30. This is for linking the variables
for w in warehouses:
    for m in markets:
        for f in fruits:
            problem_transportation += x[w][f][m] <= y[w][m] * 30




# Solve the problem
problem_transportation.solve()

# Print the results
print("Total Minimized Distance:", problem_transportation.objective.value())

print()

for m in markets:
    for w in warehouses:
        if y[w][m].value() == 1:
            print(f"Market {m} is served by Warehouse {w}")


Total Minimized Distance: 409.0

Market M1 is served by Warehouse A
Market M2 is served by Warehouse B
Market M3 is served by Warehouse A
Market M4 is served by Warehouse D
Market M5 is served by Warehouse A
Market M6 is served by Warehouse C
Market M7 is served by Warehouse D
Market M8 is served by Warehouse C
Market M9 is served by Warehouse B
Market M10 is served by Warehouse B


SUDOKU A

In [7]:

# Importing necessary functions from the PuLP library
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value

# Sudoku problemini tanımlayan fonksiyon
def solve_sudoku():
    # Sudoku problemi
    sudoku_problem = LpProblem("Sudoku_Problem", sense=LpMaximize)

    # Define the possible numbers, rows, and columns in the Sudoku puzzle
    numbers = list(range(1, 10))
    rows = list(range(1, 10))
    cols = list(range(1, 10))

    # Define decision variables as binary variables indicating the presence of a number in a cell
    sudoku_vars = LpVariable.dicts("Number", (numbers, rows, cols), cat="Binary")

    # Objective function
    sudoku_problem = LpProblem("Sudoku_Problem", sense=LpMaximize)


    # Add constraints to ensure each number appears exactly once in each row, column, and 3x3 box
    for i in rows:
        for j in cols:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for num in numbers]) == 1

    for num in numbers:
        for j in cols:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for i in rows]) == 1

    for num in numbers:
        for i in rows:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for j in cols]) == 1

    for num in numbers:
        for p in range(1, 4):
            for q in range(1, 4):
                sudoku_problem += lpSum([sudoku_vars[num][i][j] for i in range(3 * (p - 1) + 1, 3 * p + 1) for j in
                                         range(3 * (q - 1) + 1, 3 * q + 1)]) == 1


    # Specify the initial state of the Sudoku puzzle as a 9x9 grid
    initial_state = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    # Add constraints to fix the values in the initial state of the Sudoku puzzle
    for i in rows:
        for j in cols:
            val = initial_state[i - 1][j - 1]
            if val != 0:
                sudoku_problem += sudoku_vars[val][i][j] == 1

    # Solve the Sudoku problem
    sudoku_problem.solve()

    # Print the solved Sudoku puzzle
    for i in rows:
        row_values = []
        for j in cols:
            for num in numbers:
                if value(sudoku_vars[num][i][j]) == 1:
                    row_values.append(num)
        print(row_values)



# Call the solve_sudoku function to solve and print the Sudoku puzzle
solve_sudoku()


[5, 3, 4, 6, 7, 8, 9, 1, 2]
[6, 7, 2, 1, 9, 5, 3, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 5, 9, 7, 6, 1, 4, 2, 3]
[4, 2, 6, 8, 5, 3, 7, 9, 1]
[7, 1, 3, 9, 2, 4, 8, 5, 6]
[9, 6, 1, 5, 3, 7, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 4, 5, 2, 8, 6, 1, 7, 9]


SUDOKU B



In [5]:
# Importing necessary functions from the PuLP library
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value

# Function to solve a Sudoku puzzle using Linear Programming
def solve_sudoku_middlebox():

    # Create a Linear Programming problem with the objective of maximizing the sum of numbers in the middle box
    sudoku_problem = LpProblem("Sudoku_Problem", sense=LpMaximize)

    # Define the possible numbers, rows, and columns in the Sudoku puzzle
    numbers = list(range(1, 10))
    rows = list(range(1, 10))
    cols = list(range(1, 10))

    # Define decision variables as binary variables indicating the presence of a number in a cell
    sudoku_vars = LpVariable.dicts("Number", (numbers, rows, cols), cat="Binary")

    # Set the objective function to maximize the sum of numbers in the middle 3x3 box
    sudoku_problem += lpSum([sudoku_vars[num][i][j] * num for num in numbers for i in range(4, 7) for j in range(4, 7)])

    # Add constraints to ensure each number appears exactly once in each row, column
    for i in rows:
        for j in cols:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for num in numbers]) == 1

    for num in numbers:
        for j in cols:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for i in rows]) == 1

    for num in numbers:
        for i in rows:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for j in cols]) == 1

    # Specify the initial state of the Sudoku puzzle as a 9x9 grid
    initial_state = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    # Add constraints to fix the values in the initial state of the Sudoku puzzle
    for i in rows:
        for j in cols:
            val = initial_state[i - 1][j - 1]
            if val != 0:
                sudoku_problem += sudoku_vars[val][i][j] == 1

    # Solve the Sudoku problem
    sudoku_problem.solve()

    # Calculate and print the sum of numbers in the middle box
    middle_box_sum = sum([num * value(sudoku_vars[num][i][j]) for num in numbers for i in range(4, 7) for j in range(4, 7)])
    print(f"Sum of numbers in the middle box: {middle_box_sum}")

    print()
    # Print the solved Sudoku puzzle
    for i in rows:
        row_values = []
        for j in cols:
            for num in numbers:
                if value(sudoku_vars[num][i][j]) == 1:
                    row_values.append(num)
        print(row_values)

# Call the solve_sudoku function to solve and print the Sudoku puzzle
solve_sudoku_middlebox()


Sum of numbers in the middle box: 53.0

[5, 3, 9, 6, 7, 4, 8, 1, 2]
[6, 2, 3, 1, 9, 5, 7, 4, 8]
[1, 9, 8, 3, 4, 2, 5, 6, 7]
[8, 1, 2, 5, 6, 7, 4, 9, 3]
[4, 7, 6, 8, 5, 3, 9, 2, 1]
[7, 4, 1, 9, 2, 8, 3, 5, 6]
[9, 6, 5, 7, 3, 1, 2, 8, 4]
[2, 8, 7, 4, 1, 9, 6, 3, 5]
[3, 5, 4, 2, 8, 6, 1, 7, 9]


SUDOKU C

In [ ]:
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, value

# Function to solve the Sudoku problem
def solve_sudoku_nines():
    # Sudoku problem
    sudoku_problem = LpProblem("Sudoku_Problem", sense=LpMaximize)

    # Variables for a 9x9 Sudoku matrix
    numbers = list(range(1, 10))
    rows = list(range(1, 10))
    cols = list(range(1, 10))
    sudoku_vars = LpVariable.dicts("Number", (numbers, rows, cols), cat="Binary")

    # Objective function
    sudoku_problem += lpSum([sudoku_vars[9][i][j] for i in rows for j in cols])    # Each cell can have only one number

    for i in rows:
        for j in cols:
            sudoku_problem += lpSum([sudoku_vars[num][i][j] for num in numbers]) == 1

    # Constraints
    for i in rows:
        sudoku_problem += lpSum([num * sudoku_vars[num][i][j] for num in numbers for j in cols]) <= 40

    for j in cols:
        sudoku_problem += lpSum([num * sudoku_vars[num][i][j] for num in numbers for i in rows]) <= 40

    # Sudoku initial state
    initial_state = [
        [5, 3, 0, 0, 7, 0, 0, 0, 0],
        [6, 0, 0, 1, 9, 5, 0, 0, 0],
        [0, 9, 8, 0, 0, 0, 0, 6, 0],
        [8, 0, 0, 0, 6, 0, 0, 0, 3],
        [4, 0, 0, 8, 0, 3, 0, 0, 1],
        [7, 0, 0, 0, 2, 0, 0, 0, 6],
        [0, 6, 0, 0, 0, 0, 2, 8, 0],
        [0, 0, 0, 4, 1, 9, 0, 0, 5],
        [0, 0, 0, 0, 8, 0, 0, 7, 9]
    ]

    # Add the initial state
    for i in rows:
        for j in cols:
            val = initial_state[i - 1][j - 1]
            if val != 0:
                sudoku_problem += sudoku_vars[val][i][j] == 1

    # Solve the problem
    sudoku_problem.solve()

    # Print the solution
    for i in rows:
        row_values = []
        for j in cols:
            for num in numbers:
                if value(sudoku_vars[num][i][j]) == 1:
                    row_values.append(num)
        print(row_values)

    # Blank line for better readibility
    print()

    # Count the number of 9s used
    num_of_nines = sum([1 for r in rows for c in cols if value(sudoku_vars[9][r][c]) == 1])
    print("Number of digit 9 used in sudoku:", num_of_nines)


# Solve the Sudoku
solve_sudoku_nines()


[5, 3, 9, 9, 7, 1, 1, 1, 1]
[6, 1, 1, 1, 9, 5, 9, 1, 1]
[1, 9, 8, 1, 1, 9, 1, 6, 1]
[8, 9, 1, 9, 6, 1, 1, 1, 3]
[4, 1, 1, 8, 1, 3, 9, 9, 1]
[7, 9, 1, 1, 2, 1, 9, 1, 6]
[1, 6, 9, 1, 1, 1, 2, 8, 9]
[1, 1, 9, 4, 1, 9, 1, 1, 5]
[1, 1, 1, 1, 8, 9, 1, 7, 9]

Number of digit 9 used in sudoku: 18
